In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from nilearn import connectome, datasets, maskers, plotting
from src import autocorr, fmriprep

warnings.simplefilter(action="ignore", category=FutureWarning)

ex_preproc = "../tests/data/sub-c016_task-h2_space-MNI152NLin6Asym_res-3_desc-preproc_bold.nii.gz"

data = fmriprep.Data(preproc=ex_preproc, denoise_strategy="HMPWMCSFScrubGS")

atlas = datasets.fetch_atlas_schaefer_2018(n_rois=100, resolution_mm=2, yeo_networks=17)
atlas_img = atlas["maps"]
atlas_labels = [label.decode()[11:] for label in atlas["labels"]]

plotting.plot_roi(atlas_img, draw_cross=False)


#### extract mean timeseries w/in each ROI
- zscore timeseries and confounds
- `nilearn.signal.clean`: 'confounds removal is based on a projection on the orthogonal of the signal space'

In [ ]:
masker = maskers.NiftiLabelsMasker(
    atlas_img,
    atlas_labels,
    mask_img=data.mask,
    smoothing_fwhm=6,
    standardize="zscore",
    standardize_confounds=True,
    t_r=data.tr,
    strategy="mean",
    memory="tmp",
)


roi_timeseries = masker.fit_transform(
    imgs=data.preproc, confounds=data.confounds, sample_mask=data.sample_mask
)

roi_timeseries.shape


#### functional connectivity: correlation
- correlation and partial correlation

In [ ]:
def fisher_z(corr_coefficients):
    return np.arctanh(corr_coefficients)


# LedoitWolf covariance estimator
corr_model = connectome.ConnectivityMeasure(kind="correlation")
pcorr_model = connectome.ConnectivityMeasure(kind="partial correlation")

corr_coefficients = corr_model.fit_transform([roi_timeseries])[0]
pcorr_coefficients = pcorr_model.fit_transform([roi_timeseries])[0]


corr_z_transform = fisher_z(corr_coefficients)
pcorr_z_transform = fisher_z(pcorr_coefficients)

# plotting
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

kwargs = dict(vmin=-1, vmax=1, cmap="RdBu_r")

plotting.plot_matrix(corr_z_transform, axes=axs[0], colorbar=False, **kwargs)
plotting.plot_matrix(pcorr_z_transform, axes=axs[1], **kwargs)

axs[0].set_title("correlation")
axs[1].set_title("partial correlation");


#### temporal autocorrelation correction [xDF](https://github.com/asoroosh/xDF)
- regularization: ACF diminishes to zero with increasing lags
- tapering: smoothly scaling ACF estimates to zero with increasing lags

regularization methods
1. fixed truncation
2. Tukey tapering
3. adaptive scaling (which gets the most accurate estimates)

TV
- truncate to theoretical variance

xDF model
- approximates *auto correlation* and *cross correlation* using Wiener-Khinchin Theorem

In [ ]:
t = roi_timeseries.shape[0]
kwargs = dict(ts=roi_timeseries, T=t, verbose=True)

# xDF without regularization
xdf1 = autocorr.xDF_Calc(method="", TV=True, **kwargs)

# xDF without truncation
xdf2 = autocorr.xDF_Calc(method="truncate", methodparam=int(t / 4), **kwargs)

# xDF with adaptive truncation
xdf3 = autocorr.xDF_Calc(method="truncate", methodparam="adaptive", **kwargs)

# xDF without tapering
xdf4 = autocorr.xDF_Calc(method="tukey", methodparam=np.sqrt(t), **kwargs)


In [ ]:
fig, axs = plt.subplots(
    3, 5, figsize=(12, 6), gridspec_kw=dict(height_ratios=[1, 0.75, 0.75])
)
fig.tight_layout()

for idx, corr_z in enumerate(
    [xdf1["znaive"], xdf1["z"], xdf2["z"], xdf3["z"], xdf4["z"]]
):

    # multitest correction
    z_corrected = autocorr.stat_threshold(corr_z, mce="fdr_bh")[0]
    n_corrected = len(np.where(z_corrected != 0)[0]) / 2

    axs[0, idx].set_title(f"xDF model {idx}\nq < 0.05: {n_corrected}")

    # plot z-transformed correlation matrix
    corr_z[np.diag_indices_from(corr_z)] = np.nan
    plotting.plot_matrix(corr_z, axes=axs[0, idx], vmin=-15, vmax=15, colorbar=False)

    # add kde plot
    sns.distplot(corr_z.reshape(-1), ax=axs[1, idx], kde=False)

    # corrected vs uncorrected scatter plot
    sns.scatterplot(
        corr_z.reshape(-1), xdf1["znaive"].reshape(-1), ax=axs[2, idx], s=0.5
    )

axs[2, 0].set_xlabel("xDF adjusted Z-score")
axs[2, 0].set_ylabel("uncorrected Z-score")
fig.tight_layout()


#### functional connectivity: PPI

In [ ]:
dm = data.make_design_matrix(hrf_model="glover + derivative", drop_constant=True)
dm.plot(subplots=True, figsize=(20, 2))


In [ ]:
roi_ts_sample = pd.DataFrame(
    roi_timeseries, columns=atlas_labels, index=dm.index
).sample(n=3, random_state=5, axis="columns")

sns.pairplot(
    roi_ts_sample.join(dm["stim"]), height=1.5, plot_kws=dict(size=1, alpha=0.5)
)


#### linear model summary

In [ ]:
lm = smf.ols(
    formula="LH_DefaultB_PFCv_2 ~ (stim * LH_ContA_PFCl_2) + stim_derivative",
    data=dm.join(roi_ts_sample),
).fit()
print(lm.summary())


#### model diagnostics: residual plots

In [ ]:
lm.resid.plot(figsize=(20, 1), title="residuals")


In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 4))
sns.regplot(lm.fittedvalues, lm.resid, lowess=True, ax=axs[0])
axs[0].set_xlabel("Fitted Values")
axs[0].set_ylabel("Residuals")
sm.qqplot(lm.resid, line="s", ax=axs[1]);


#### temporal autocorrelation

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 4))
pd.plotting.lag_plot(lm.resid, lag=1, ax=axs[0])
pd.plotting.autocorrelation_plot(lm.resid, ax=axs[1])
